# Baseline Flat Classification

This notebook trains a **flat (non-hierarchical) classifier** on the merged 3D MedMNIST datasets for comparison with the hierarchical approach.

The flat classifier predicts the fine-grained label directly without the intermediate region classification step.

In [6]:
import sys
sys.path.insert(0, '..')

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from medmnist import INFO

from config import (
    DEVICE, DATA_CONFIG, MODEL_CONFIG, TRAINING_CONFIG,
    PATHS, set_seed, DEFAULT_MERGED_DATASETS
)
from utils.data_loader import create_hierarchical_dataset
from utils.cnn_3d_models import get_3d_model
from utils.trainer import Trainer
from utils.metrics import compute_metrics

set_seed(42)
print(f"Device: {DEVICE}")

Device: cpu


## 1. Load Merged Dataset

In [ ]:
# Load all 5 merged datasets
print(f"Loading datasets: {DEFAULT_MERGED_DATASETS}")

train_loader, val_loader, test_loader, dataset_info = create_hierarchical_dataset(
    datasets_to_include=DEFAULT_MERGED_DATASETS,
    batch_size=DATA_CONFIG['batch_size'],
    num_workers=DATA_CONFIG['num_workers'],
    return_global_labels=True 
)

print("="*60)
print("MERGED DATASET INFO")
print("="*60)
# 1. Basic info table
basic_info = pd.DataFrame([{
    'Total Fine Classes': dataset_info['num_fine_classes'],
    'Train Samples': dataset_info['train_samples'],
    'Val Samples': dataset_info['val_samples'],
    'Test Samples': dataset_info['test_samples']
}])
print("\nDataset Overview:")
display(
    basic_info.style
    .hide(axis="index")
    .set_properties(**{'text-align': 'center'})
)
# 2. Global Fine Label Mapping for Flat Classifier
print("\nGlobal Fine Label Mapping (for Flat Classifier):")
rows = []
global_map = dataset_info['global_idx_to_name']

for global_idx in sorted(global_map.keys()):
    info = global_map[global_idx]
    rows.append({
        'Global Index': int(global_idx),
        'Region': info['region'].upper(),
        'Dataset': info['dataset'].upper(),
        'Label Name': info['name']
    })
df_labels = pd.DataFrame(rows)
# Display with specific alignment
display(
    df_labels.style
    .hide(axis="index")
    .set_properties(subset=['Global Index', 'Region', 'Dataset'], **{'text-align': 'center'})
    .set_properties(subset=['Label Name'], **{'text-align': 'left'})
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'th.col3', 'props': [('text-align', 'left')]} # Left align 'Label Name' header
    ])
)


Loading datasets: ['organ', 'nodule', 'adrenal', 'fracture', 'vessel']
MERGED DATASET INFO

Dataset Overview:


Total Fine Classes,Train Samples,Val Samples,Test Samples
20,5679,718,1840



Global Fine Label Mapping (for Flat Classifier):


Global Index,Region,Dataset,Label Name
0,ABDOMEN,ADRENAL,normal
1,ABDOMEN,ADRENAL,hyperplasia
2,ABDOMEN,ORGAN,liver
3,ABDOMEN,ORGAN,kidney-right
4,ABDOMEN,ORGAN,kidney-left
5,ABDOMEN,ORGAN,femur-right
6,ABDOMEN,ORGAN,femur-left
7,ABDOMEN,ORGAN,bladder
8,ABDOMEN,ORGAN,spleen
9,ABDOMEN,ORGAN,pancreas


## 2. Create Wrapper DataLoaders for Flat Training

The hierarchical dataset returns `(img, coarse_label, fine_label)`. For flat training, we only need `(img, fine_label)`.

In [8]:
class FlatDataLoaderWrapper:
    """Wrapper that extracts only (img, global_fine_label) from hierarchical dataloader."""
    def __init__(self, hierarchical_loader):
        self.loader = hierarchical_loader
    
    def __iter__(self):
        # Unpack 4 values: img, coarse, fine_local, fine_global
        for imgs, coarse_labels, fine_labels, global_fine_labels in self.loader:
            # Yield image and GLOBAL label (0-19)
            yield imgs, global_fine_labels
    
    def __len__(self):
        return len(self.loader)

# Create flat loaders
flat_train_loader = FlatDataLoaderWrapper(train_loader)
flat_val_loader = FlatDataLoaderWrapper(val_loader)
flat_test_loader = FlatDataLoaderWrapper(test_loader)

print("\nFlat data loaders created")


Flat data loaders created


## 3. Build Flat Classifier

In [9]:
# Create flat classifier
num_classes = dataset_info['num_fine_classes']
architecture = MODEL_CONFIG['architecture']

# Instantiate model
model = get_3d_model(
    model_name=architecture,
    num_classes=num_classes
).to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nFlat classifier created:")

# Create summary table
model_summary = pd.DataFrame([{
    'Architecture': architecture,
    'Input Channels': 1,  # Assuming grayscale medical images
    'Output Classes': num_classes,
    'Total Parameters': f"{total_params:,}",
    'Trainable Parameters': f"{trainable_params:,}",
    'Device': str(DEVICE)
}])
# Display styled table
display(
    model_summary.style
    .hide(axis="index")
    .set_properties(**{'text-align': 'center'})
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]}
    ])
)


Flat classifier created:


Architecture,Input Channels,Output Classes,Total Parameters,Trainable Parameters,Device
enhanced,1,20,"2,097,460","2,097,460",cpu


## 4. Train Flat Classifier

In [10]:
# Create trainer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=TRAINING_CONFIG['weight_decay']
)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=TRAINING_CONFIG['scheduler_step_size'],
    gamma=TRAINING_CONFIG['scheduler_gamma']
)

trainer = Trainer(
    model=model,
    train_loader=flat_train_loader,
    val_loader=flat_val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler
)

# Total epochs = coarse + fine epochs for fair comparison
total_epochs = TRAINING_CONFIG['coarse_epochs'] + TRAINING_CONFIG['fine_epochs']
print(f"Training for {total_epochs} epochs...")

Training for 50 epochs...


In [11]:
# Train
history = trainer.train(num_epochs=total_epochs)

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)


Epoch 1/50


Training:   0%|          | 0/710 [00:00<?, ?it/s]c:\Users\BS-06\Desktop\HMIC\HierarchicalMIC\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Training:  27%|██▋       | 193/710 [00:42<01:53,  4.57it/s, loss=2.24, acc=0.344] 


KeyboardInterrupt: 

## 5. Training Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', color='blue')
axes[0].plot(history['val_loss'], label='Val Loss', color='orange')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Flat Classifier Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc', color='blue')
axes[1].plot(history['val_acc'], label='Val Acc', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Flat Classifier Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{PATHS['figures']}/baseline_training_{architecture}.png", dpi=150)
plt.show()

## 6. Evaluation on Test Set

In [ ]:
def evaluate_flat_model(model, test_loader, device, dataset_info):
    """Evaluate a flat (non-hierarchical) model on the test set and compute metrics."""
    from utils.metrics import compute_metrics
    from medmnist import INFO

    model.eval()
    all_preds = []
    all_labels = []

    # Run inference on the test set
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Evaluating"):
            imgs = imgs.to(device, dtype=torch.float32)

            # Normalize if images are in [0, 255]
            if imgs.max() > 1:
                imgs = imgs / 255.0
            # Convert labels to shape [B] and move to device
            labels = labels.squeeze(-1).long().to(device)
            # Forward pass and prediction
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)

            # Accumulate predictions and ground truth
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Placeholder label map (index → class name), can be filled using INFO if needed
    label_map = {idx: "" for idx in range(dataset_info['num_fine_classes'])}
    
    # Compute evaluation metrics using accumulated predictions
    metrics = compute_metrics(all_labels, all_preds)

    return metrics


In [ ]:
# Evaluate
results = evaluate_flat_model(model, flat_test_loader, DEVICE, dataset_info)
print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Macro F1: {results['f1_score']:.4f}")
print(f"Weighted Recall: {results['recall']:.4f}")
print("\nClassification Report (Per-Class Performance):")

# Re-generate report with named classes if possible, currently using IDs
print(results.get('classification_report', "Report not generated in utils"))

# Plot Confusion Matrix
plt.figure(figsize=(12, 10))
sns.heatmap(results['confusion_matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Ground Truth')
plt.show()

## 7. Save Model

In [ ]:
import os
os.makedirs(PATHS['models'], exist_ok=True)

model_path = f"{PATHS['models']}/baseline_{architecture}.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'dataset_info': dataset_info,
    'history': history,
    'test_results': results,
    'config': {
        'architecture': architecture,
        'num_classes': num_classes,
    }
}, model_path)

print(f"\n✓ Model saved to: {model_path}")

## 8. Summary

In [ ]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"\nDatasets used: {', '.join(dataset_info['datasets_included'])}")
print(f"Total training samples: {dataset_info['train_samples']:,}")
print(f"\nModel architecture: {architecture}")
print(f"Number of classes: {num_classes}")
print(f"Total parameters: {total_params:,}")
print(f"\nFinal Test Accuracy: {results['accuracy']:.4f}")
print(f"\nModel saved to: {model_path}")